# Final Project – Linear Optimization (Simplex Method)

**Course:** Optimization  
**Student name:** *Diego Navarro Miranda*  
**Student ID:** *2131979*  

In this project we model and solve a real-life linear optimization problem using:

- Mathematical modeling  
- Graphical method  
- Simplex method (via a Python solver)  
- Validation and analysis of results  

All data used in this notebook is stored in the file `coffee_data.csv`, which is submitted together with this notebook.


In [ ]:
## 1. Problem Description

We study a small **coffee roastery** that produces two types of roasted coffee:

- **Premium Blend (x₁)**  
- **House Blend (x₂)**  

Each kilogram of coffee requires **roasting time** in an industrial roaster and **packaging time** in a packing machine.  
The company wants to decide how many kilograms of each blend to produce in one day in order to **maximize the profit**.

### Available resources

- The roaster is available for at most **720 minutes per day**.  
- The packaging machine is available for at most **300 minutes per day**.

### Processing times and profit

From historical data (stored in `coffee_data.csv`):

- Premium Blend  
  - Profit: 4 USD per kg  
  - Roasting time: 12 minutes per kg  
  - Packaging time: 4 minutes per kg  
  - Maximum daily demand: 100 kg  

- House Blend  
  - Profit: 3 USD per kg  
  - Roasting time: 6 minutes per kg  
  - Packaging time: 3 minutes per kg  
  - Maximum daily demand: 150 kg  

### Motivation (Real-life)

This problem is inspired by real small roasting businesses.  
They usually have:

- Limited machine time (roasters + packaging).  
- Different profit margins for each product.  
- Demand limits: producing more than customers want is risky and expensive.

Optimizing production helps the company:

- Use the machines more efficiently.  
- Focus on the most profitable products.  
- Increase daily profit while respecting all operational limits.

This goes beyond a standard textbook example, because it is based on a realistic specialty-coffee scenario.


In [ ]:
## 2. Mathematical Formulation

### Decision Variables

\[
x_V = \text{number of Vegan Boxes produced per week}
\]
\[
x_P = \text{number of Protein Boxes produced per week}
\]

Both variables are continuous and nonnegative:
\[
x_V \ge 0,\quad x_P \ge 0
\]

### Objective Function

Maximize weekly profit:

\[
\max z = 12x_V + 18x_P
\]

### Constraints

1. **Vegetable limit**

\[
1.5x_V + 1x_P \le 90
\]

2. **Grain limit**

\[
0.5x_V \le 80
\]

3. **Protein limit**

\[
1x_P \le 70
\]

4. **Packaging time**

60 hours = 3,600 minutes

\[
6x_V + 8x_P \le 3600
\]

5. **Minimum total production**

\[
x_V + x_P \ge 100
\]

6. **Maximum demand for Protein Box**

\[
x_P \le 120
\]

7. **Nonnegativity**

\[
x_V \ge 0,\quad x_P \ge 0
\]

This is a linear programming model: the objective function and all constraints are linear.


In [1]:
!pip install pulp


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# 1. Import libraries
# If PuLP is not installed in your environment, uncomment the next line:
# !pip install pulp

import pandas as pd
import pulp as pl

# 2. Create a small data table with the problem parameters
products = ["Vegan Box", "Protein Box"]

profit = {
    "Vegan Box": 12,
    "Protein Box": 18,
}

veg_usage = {
    "Vegan Box": 1.5,
    "Protein Box": 1.0,
}

grain_usage = {
    "Vegan Box": 0.5,
    "Protein Box": 0.0,
}

protein_usage = {
    "Vegan Box": 0.0,
    "Protein Box": 1.0,
}

time_usage = {
    "Vegan Box": 6,
    "Protein Box": 8,
}

# Resource limits
veg_limit = 90
grain_limit = 80
protein_limit = 70
time_limit = 3600  # minutes
max_protein_boxes = 120
min_total_boxes = 100

# 3. Put the data into a DataFrame
data = pd.DataFrame({
    "Product": products,
    "Profit_per_box": [profit[p] for p in products],
    "Veg_kg": [veg_usage[p] for p in products],
    "Grain_kg": [grain_usage[p] for p in products],
    "Protein_kg": [protein_usage[p] for p in products],
    "Time_minutes": [time_usage[p] for p in products],
})

data


,Product,Profit_per_box,Veg_kg,Grain_kg,Protein_kg,Time_minutes
0,Vegan Box,12,1.5,0.5,0.0,6
1,Protein Box,18,1.0,0.0,1.0,8


In [18]:
data.to_csv("mealbox_data.csv", index=False)

print("Data file 'mealbox_data.csv' has been created in the current folder.")


Data file 'mealbox_data.csv' has been created in the current folder.


In [4]:
# 4. Read data from the CSV file
import pandas as pd

data = pd.read_csv("mealbox_data.csv")
print("Data used in the optimization model:")
display(data)


Data used in the optimization model:


,Product,Profit_per_box,Veg_kg,Grain_kg,Protein_kg,Time_minutes
0,Vegan Box,12,1.5,0.5,0.0,6
1,Protein Box,18,1.0,0.0,1.0,8


In [5]:
# 5. Build the optimization model

import pulp as pl

# Create the model (maximization)
model = pl.LpProblem("MealBox_Production_Planning", pl.LpMaximize)

# Decision variables: number of boxes for each product
x = pl.LpVariable.dicts(
    "Boxes", 
    data["Product"], 
    lowBound=0,  # non-negativity
    cat="Continuous"
)

# ----- Objective function: maximize total profit -----
model += pl.lpSum(
    row.Profit_per_box * x[row.Product]
    for row in data.itertuples()
), "Total_Profit"

# Resource limits (taken from your previous cell)
veg_limit = 90      # kg of vegetables available
grain_limit = 80    # kg of grains available
protein_limit = 70  # kg of protein available
time_limit = 3600   # minutes of packaging time
max_protein_boxes = 120
min_total_boxes = 100

# ----- Constraints -----

# 1) Vegetable limit
model += pl.lpSum(
    row.Veg_kg * x[row.Product]
    for row in data.itertuples()
) <= veg_limit, "Vegetable_limit"

# 2) Grain limit
model += pl.lpSum(
    row.Grain_kg * x[row.Product]
    for row in data.itertuples()
) <= grain_limit, "Grain_limit"

# 3) Protein limit
model += pl.lpSum(
    row.Protein_kg * x[row.Product]
    for row in data.itertuples()
) <= protein_limit, "Protein_limit"

# 4) Packaging time limit
model += pl.lpSum(
    row.Time_minutes * x[row.Product]
    for row in data.itertuples()
) <= time_limit, "Time_limit"

# 5) Minimum total production
model += pl.lpSum(
    x[row.Product] for row in data.itertuples()
) >= min_total_boxes, "Minimum_total_boxes"

# 6) Maximum demand for Protein Box
model += x["Protein Box"] <= max_protein_boxes, "Max_protein_boxes"


In [6]:
# 6. Solve the model

solution_status = model.solve(pl.PULP_CBC_CMD(msg=False))

print("Solution status:", pl.LpStatus[solution_status])


Solution status: Infeasible


In [16]:
# 7. Show optimal decision variables and total profit

results = []

for row in data.itertuples():
    product = row.Product
    quantity = x[product].value()
    results.append([product, quantity])

results_df = pd.DataFrame(results, columns=["Product", "Optimal_boxes"])
display(results_df)

# Total profit
total_profit = pl.value(model.objective)
print(f"Total optimal profit = ${total_profit:0.2f}")


,Product,Optimal_boxes
0,Vegan Box,13.333333
1,Protein Box,70.000000


Total optimal profit = $1420.00


In [17]:
# 8. Validation: check how much of each resource is used

veg_used = sum(row.Veg_kg * x[row.Product].value() for row in data.itertuples())
grain_used = sum(row.Grain_kg * x[row.Product].value() for row in data.itertuples())
protein_used = sum(row.Protein_kg * x[row.Product].value() for row in data.itertuples())
time_used = sum(row.Time_minutes * x[row.Product].value() for row in data.itertuples())
total_boxes = sum(x[row.Product].value() for row in data.itertuples())

validation_table = pd.DataFrame({
    "Resource": ["Vegetables (kg)", "Grains (kg)", "Protein (kg)", "Time (minutes)", "Total boxes"],
    "Used":     [veg_used, grain_used, protein_used, time_used, total_boxes],
    "Limit":    [veg_limit, grain_limit, protein_limit, time_limit, ""]
})

display(validation_table)


,Resource,Used,Limit
0,Vegetables (kg),90.000000,90
1,Grains (kg),6.666666,80
2,Protein (kg),70.000000,70
3,Time (minutes),639.999998,3600
4,Total boxes,83.333333,
